# Sparkify - Data Lake using Spark

### General Imports

#### Standard

In [1]:
import os
import logging
import sys
import configparser
from datetime import datetime

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4,application_1589142924251_0005,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Boto3

In [3]:
sc.install_pypi_package('boto3')
import boto3
from botocore.exceptions import ClientError

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached boto3-1.13.6-py2.py3-none-any.whl (128 kB)
  Using cached s3transfer-0.3.3-py2.py3-none-any.whl (69 kB)
  Using cached botocore-1.16.6-py2.py3-none-any.whl (6.2 MB)
  Using cached docutils-0.15.2-py3-none-any.whl (547 kB)
  Using cached urllib3-1.25.9-py2.py3-none-any.whl (126 kB)
  Using cached python_dateutil-2.8.1-py2.py3-none-any.whl (227 kB)

#### PySpark

In [4]:
from pyspark.sql import SparkSession, Window
from  pyspark.sql.dataframe import DataFrame
from pyspark.sql.functions import (udf, col, year, month, dayofmonth, hour,
    weekofyear, date_format, dayofweek, max, monotonically_increasing_id)
from pyspark.sql.types import (
    StructType, StructField, StringType, DoubleType, IntegerType, TimestampType)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Deployment Stage and Application Name

In [5]:
APP = 'sparkify'
STAGE = 'dev'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Setting Up Logging

In [ ]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.addHandler(logging.StreamHandler())

In [ ]:
logger.info('aa')

### Setting up AWS Env Variables

In [ ]:
def load_config_file(filename:str):
    config = configparser.ConfigParser()
    config.read(filename)
    print(config)
    return config

In [ ]:
config_filename = './dl.cfg'
config = load_config_file(config_filename)

### Create Bucket to Store Data

In [ ]:
def create_bucket(bucket_name:str, region:str=os.getenv('AWS_DEFAULT_REGION'), acl="private"):
    """
    Creates a bucket name in the specified region
    
    Boto3 will make use of the set environment variables or assumed role.
    
    Args:
        bucket_name {str} -- Bucket to create
        region {str} -- String region to create bucket in, defaults to AWS_DEFAULT_REGION in env variable
        acl {str} -- Canned access control list to apply to the bucket. 'public-read'
            makes sure everything posted is publicly readable
    Returns:
        {str} -- bucket_name
    """
    
    assert region, "No region was defined"
    
    # Create bucket
    try:
        s3_client = boto3.client('s3', region_name=region)
        location = {'LocationConstraint': region}
        s3_client.create_bucket(
            Bucket=bucket_name,
            CreateBucketConfiguration=location
            )
            
        logger.info(f"Bucket `{bucket_name}` Created")
    except ClientError as e:
        logging.exception("Could not create bucket")
        raise e
        
    return bucket_name

In [6]:
output_bucket_name = f'{APP}-{STAGE}'
# create_bucket(output_bucket_name)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Create Spark Session

In [ ]:
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.amazonaws:aws-java-sdk-pom:1.10.34,org.apache.hadoop:hadoop-aws:2.7.2 pyspark-shell'
def create_spark_session():
    """Create a Spark session"""
    
    spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()
    return spark

In [ ]:
spark = create_spark_session()

### Raw Data Location

In [12]:
song_data_path =  "s3a://udacity-dend/song_data" #/A/A/A
log_data_path = "s3a://udacity-dend/log_data" #2018/11"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### General Spark DF Classes

In [7]:
class SparkDF(object):
    def __init__(self, spark_session):
        self.spark = spark_session
    
    def _load_json_data(self):
        location = f"{self.location}/*.json"
        return self.spark.read.json(location, schema=self.schema)
    
    def _write_to_parquet(self, s3_output_path:str, mode:str='overwrite', partitions:list=[]):
        self.df.write.parquet(
            s3_output_path,
            mode=mode,
            partitionBy=partitions
        )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
class RawDF(SparkDF):
    def __init__(self,spark_session, location:str, data_schema:StructType):
        super().__init__(spark_session)
        self.location = location
        self.schema = data_schema
        self.df = self._load_json_data()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
class DerivativeDF(SparkDF):
    def __init__(self, df:DataFrame):
        self.df = df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Process Song Data

#### Song Spark Schema

In [10]:
def get_song_schema():
    return StructType([
            StructField("artist_id", StringType(), False),
            StructField("artist_latitude", StringType(), True),
            StructField("artist_longitude", StringType(), True),
            StructField("artist_location", StringType(), True),
            StructField("artist_name", StringType(), False),
            StructField("song_id", StringType(), False),
            StructField("title", StringType(), False),
            StructField("duration", DoubleType(), False),
            StructField("year", IntegerType(), False)
    ])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
song_raw = RawDF(spark,f"{song_data_path}/*/*/*", get_song_schema())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Create Songs Table

In [14]:
songs = DerivativeDF(song_raw.df.select("song_id", "title", "artist_id", "year", "duration"))    

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
songs._write_to_parquet(
    s3_output_path=f"s3://{output_bucket_name}/songs",
    partitions=["year", "artist_id"]
    )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Create Artists Table

In [16]:
artists = DerivativeDF(song_raw.df
        .select(
            "artist_id",
            col("artist_name").alias("name"),
            col("artist_location").alias("location"),
            col("artist_latitude").alias("latitude"),
            col("artist_longitude").alias("longitude"))
        .distinct()
    )
artists._write_to_parquet(
    s3_output_path=f"s3://{output_bucket_name}/artists",
    partitions=["artist_id"]
    )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Process Log Data

#### Log Spark Schema

In [17]:
def get_log_schema():
    return StructType([
        StructField("artist", StringType(), True),
        StructField("auth", StringType(), False),
        StructField("firstName", StringType(), True),
        StructField("gender", StringType(), True),
        StructField("itemInSession", IntegerType(), False),
        StructField("lastName", StringType(), True),
        StructField("length", DoubleType(), True),
        StructField("level", StringType(), False),
        StructField("location", StringType(), True),
        StructField("method", StringType(), False),
        StructField("page", StringType(), False),
        StructField("registration", DoubleType(), True),
        StructField("sessionId", IntegerType(), False),
        StructField("song", StringType(), True),
        StructField("status", IntegerType(), False),
        StructField("ts", DoubleType(), False),
        StructField("userAgent", StringType(), True),
        StructField("userId", StringType(), True)
    ])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Create UDF to add timestamp

In [18]:
get_datetime = udf(
        lambda x: datetime.fromtimestamp(x / 1000).replace(microsecond=0),
        TimestampType()
    )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
log_raw = RawDF(spark, f"{log_data_path}/*/*", get_log_schema())
log_raw.df = log_raw.df.filter(col("page") == "NextSong")
log_raw.df = log_raw.df.withColumn("start_time", get_datetime("ts"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Create Users Table

In [20]:
users = DerivativeDF(log_raw.df
        .withColumn("max_ts_user", max("ts").over(Window.partitionBy("userID")))
        .filter(
            (col("ts") == col("max_ts_user")) &
            (col("userID") != "") &
            (col("userID").isNotNull())
        )
        .select(
            col("userID").alias("user_id"),
            col("firstName").alias("first_name"),
            col("lastName").alias("last_name"),
            "gender",
            "level"
        )
    )
users._write_to_parquet(
    s3_output_path=f"s3://{output_bucket_name}/users",
    partitions=["level"]
    )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Create Time Table

In [21]:
time = DerivativeDF(log_raw.df
        .withColumn("hour", hour("start_time"))
        .withColumn("day", dayofmonth("start_time"))
        .withColumn("week", weekofyear("start_time"))
        .withColumn("month", month("start_time"))
        .withColumn("year", year("start_time"))
        .withColumn("weekday", dayofweek("start_time"))
        .select("start_time", "hour", "day", "week", "month", "year", "weekday")
        .distinct()
         )
time._write_to_parquet(
    s3_output_path=f"s3://{output_bucket_name}/time",
    partitions=["year", "month"]
    )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Create SongsPlay Table

In [22]:
songs_temp = (
        songs.df
        .join(artists.df, "artist_id", "full")
        .select("song_id", "title", "artist_id", "name", "duration")
)

songplays_temp = log_raw.df.join(
        songs_temp,
        [
            log_raw.df.song == songs_temp.title,
            log_raw.df.artist == songs_temp.name,
            log_raw.df.length == songs_temp.duration
        ],
        "left"
)

songplays = DerivativeDF(songplays_temp
        .join(time.df, "start_time", "left")
        .select(
            "start_time",
            col("userId").alias("user_id"),
            "level",
            "song_id",
            "artist_id",
            col("sessionId").alias("session_id"),
            "location",
            col("userAgent").alias("user_agent"),
            "year",
            "month"
        )
        .withColumn("songplay_id", monotonically_increasing_id())
)

songplays._write_to_parquet(
    s3_output_path=f"s3://{output_bucket_name}/songplays",
    partitions=["year", "month"]
    )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Run Full ETL

In [ ]:
def process_song_data(spark, s3_raw_data_path:str, output_bucket_name:str):
    
    
    def get_song_schema():
        return StructType([
                StructField("artist_id", StringType(), False),
                StructField("artist_latitude", StringType(), True),
                StructField("artist_longitude", StringType(), True),
                StructField("artist_location", StringType(), True),
                StructField("artist_name", StringType(), False),
                StructField("song_id", StringType(), False),
                StructField("title", StringType(), False),
                StructField("duration", DoubleType(), False),
                StructField("year", IntegerType(), False)
        ])

    def create_song_table(song_raw, output_bucket_name:str):
        songs = DerivativeDF(song_raw.df.select("song_id", "title", "artist_id", "year", "duration"))
        songs._write_to_parquet(
            s3_output_path=f"s3://{output_bucket_name}/songs",
            partitions=["year", "artist_id"]
            )
        
        return songs
    
    
    def create_artists_table(song_raw, output_bucket_name:str):
        artists = DerivativeDF(song_raw.df
            .select(
                "artist_id",
                col("artist_name").alias("name"),
                col("artist_location").alias("location"),
                col("artist_latitude").alias("latitude"),
                col("artist_longitude").alias("longitude"))
            .distinct()
        )
        artists._write_to_parquet(
            s3_output_path=f"s3://{output_bucket_name}/artists",
            partitions=["artist_id"]
            )
        
        return artists
    
    logger.info(f"Reading and Processing `{s3_raw_data_path}`")
    song_raw = RawDF(spark, s3_raw_data_path, get_song_schema())
    logger.info("Processing and writting `songs` data")
    songs = create_song_table(song_raw, output_bucket_name)
    logger.info("Processing and writting `artists` data")
    artists = create_artists_table(song_raw, output_bucket_name)
    
    return songs, artists

In [ ]:
def process_log_data(spark, s3_raw_data_path:str, output_bucket_name:str, songs:DerivativeDF, artists:DerivativeDF):
    
    def get_log_schema():
        return StructType([
            StructField("artist", StringType(), True),
            StructField("auth", StringType(), False),
            StructField("firstName", StringType(), True),
            StructField("gender", StringType(), True),
            StructField("itemInSession", IntegerType(), False),
            StructField("lastName", StringType(), True),
            StructField("length", DoubleType(), True),
            StructField("level", StringType(), False),
            StructField("location", StringType(), True),
            StructField("method", StringType(), False),
            StructField("page", StringType(), False),
            StructField("registration", DoubleType(), True),
            StructField("sessionId", IntegerType(), False),
            StructField("song", StringType(), True),
            StructField("status", IntegerType(), False),
            StructField("ts", DoubleType(), False),
            StructField("userAgent", StringType(), True),
            StructField("userId", StringType(), True)
        ])
    
    get_datetime = udf(
        lambda x: datetime.fromtimestamp(x / 1000).replace(microsecond=0),
        TimestampType()
    )
    
    def create_users_table(log_raw, output_bucket_name:str):
        users = DerivativeDF(log_raw.df
                        .withColumn("max_ts_user", max("ts").over(Window.partitionBy("userID")))
                        .filter(
                            (col("ts") == col("max_ts_user")) &
                            (col("userID") != "") &
                            (col("userID").isNotNull())
                        )
                        .select(
                            col("userID").alias("user_id"),
                            col("firstName").alias("first_name"),
                            col("lastName").alias("last_name"),
                            "gender",
                            "level"
                        )
            )
        users._write_to_parquet(
            s3_output_path=f"s3://{output_bucket_name}/users",
            partitions=["level"]
            )
        
        return users
    
    def create_time_table(log_raw, output_bucket_name:str):
        time = DerivativeDF(log_raw.df
                            .withColumn("hour", hour("start_time"))
                            .withColumn("day", dayofmonth("start_time"))
                            .withColumn("week", weekofyear("start_time"))
                            .withColumn("month", month("start_time"))
                            .withColumn("year", year("start_time"))
                            .withColumn("weekday", dayofweek("start_time"))
                            .select("start_time", "hour", "day", "week", "month", "year", "weekday")
                            .distinct()
                             )
        
        time._write_to_parquet(
            s3_output_path=f"s3://{output_bucket_name}/time",
            partitions=["year", "month"]
            )
        
        return time
        
    
    def create_songsplay_table(log_raw, songs, artists, output_bucket_name:str):
        songs_temp = (
                songs.df
                .join(artists.df, "artist_id", "full")
                .select("song_id", "title", "artist_id", "name", "duration")
        )

        songplays_temp = log_raw.df.join(
                songs_temp,
                [
                    log_raw.df.song == songs_temp.title,
                    log_raw.df.artist == songs_temp.name,
                    log_raw.df.length == songs_temp.duration
                ],
                "left"
        )

        songplays = DerivativeDF(songplays_temp
                .join(time.df, "start_time", "left")
                .select(
                    "start_time",
                    col("userId").alias("user_id"),
                    "level",
                    "song_id",
                    "artist_id",
                    col("sessionId").alias("session_id"),
                    "location",
                    col("userAgent").alias("user_agent"),
                    "year",
                    "month"
                )
                .withColumn("songplay_id", monotonically_increasing_id())
        )
        
        songplays._write_to_parquet(
            s3_output_path=f"s3://{output_bucket_name}/songplays",
            partitions=["year", "month"]
            )
        
        return songplays


    logger.info(f"Reading and Processing `{s3_raw_data_path}`")
    log_raw = RawDF(spark, s3_raw_data_path, get_log_schema())
    log_raw.df = log_raw.df.filter(col("page") == "NextSong")
    log_raw.df = log_raw.df.withColumn("start_time", get_datetime("ts"))
    
    logger.info("Processing and writting `users` data")
    users = create_users_table(log_raw, output_bucket_name)
    logger.info("Processing and writting `time` data")
    time = create_time_table(log_raw, output_bucket_name)
    logger.info("Processing and writting `songplays` data")
    songplays = create_songsplay_table(log_raw, songs, artists, output_bucket_name)
    
    return users, time, songplays

In [ ]:
def run_sparkify_etl(APP:str='sparkify', STAGE:str='dev', bucket_exists:bool=True):
    # If running on EMR cluster, name spark is available from the get-go
#     try:
#         spark
#     except:
#         spark = create_spark_session()
    
    raw_data_bucket_name = "udacity-dend"
    song_data_path =  f"s3a://{raw_data_bucket_name}/song_data/*/*/*"
    log_data_path = f"s3a://{raw_data_bucket_name}/log_data/*/*"
    
    song_data_path =  f"s3a://{raw_data_bucket_name}/song_data/*/*/*"
    log_data_path = f"s3a://{raw_data_bucket_name}/log_data/*/*"
    
    output_bucket_name =  f'{APP}-{STAGE}'
    if not bucket_exists:
        logger.info(f"Creating Bucket: `{output_bucket_name}`")
        create_bucket(output_bucket_name)
    
    logger.info(f"Running Sparkigy ETL.\n \
                  Reading raw data from `{raw_data_bucket_name}` \n\
                  Writting output to `{output_bucket_name}`"
               )
    logger.info("Processing Song Data")
    songs, artists = process_song_data(spark, song_data_path, output_bucket_name)
    logger.info("Processing Log Data")
    users, time, songplays = process_log_data(spark, log_data_path, output_bucket_name, songs, artists)
    logger.info("Sparkify ETL is completed")

In [ ]:
run_sparkify_etl()